# Linear Equations and Computer Basics

This is the first in a set of Notebooks that follow along with the book [*Applied Computational Economics and Finance*](https://mitpress.mit.edu/books/applied-computational-economics-and-finance) (Miranda & Fackler 2004).  The goal is to actually perform some of the exercises and examples that appear througout the book to help solidify my comprehension.  It obviously starts simple, but let's see where we end up.

In [1]:
using Distributions

## LU Factorization (2.1)

The first example is actually not LU, but forward substitution.  The idea being that it is a completely reasonable method (from a computational standpoint) when we are dealing with a triangular matrix.  Suppose we have a linear model:

$$Ax=b$$

where the following dimensions apply:

+ $A (n \times p)$
+ $x (p \times 1)$
+ $b (n \times 1)$

$A$ and $b$ are given, and $A$ is a lower triangular matrix. 

$$A=
\begin{bmatrix}
a_{11} &         &        &           & 0  \\
a_{21} & a_{22} &        &           &    \\
a_{31} & a_{32} & \ddots &           &    \\
\vdots  & \vdots  & \ddots & \ddots    &    \\
a_{n1} & a_{n2} & \ldots & a_{nn-1} & a_{nn}
\end{bmatrix}$$

How does one determine the solution vector $x$?  Let's try this forward substitution business out.  The algorithm solves for $x$ one position at a time.  It can do so because the equation in the first row only has one unknown.  

$$a_{11}x_1=b_1$$

The second row equation has two unknowns, but one of them was calculated in the first row.  

$$a_{21}x_1 + a_{22}x_2 = b_2$$

Each new row introduces only one new unknown.

$$x_1 = b_1/a_{11}$$

$$x_2=(b_2-a_{21}x_1)/a_{22}$$

$$x_3=(b_3-a_{31}x_1-a_{32}x_2)/a_{33}$$

$$\vdots$$

$$x_n=(b_n - a_{n1}x_1 - a_{n2}x_2 - \cdots - a_{n,n-1}x_{n-1})/a_{nn}$$

This progression generalizes to the following expression

$$x_i = (b_i - \sum_{j=1}^{i-1} a_{ij} x_j)/a_{ii}$$

In [23]:
#Define extent of solution vector
n=5

#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

#Coerce it into a lower triangular by zero sub
for i=1:n
    for j=1:n
        if j>i
            A[i,j]=0
        end
    end
end

#Generate a response vector of random positive ints
b=float(rand(1:10,n))

# Generate array to hold solution
# x=[0. for i=1:n]

#Define function to solve this system by forward substitution
function fsub(A,b)
    #Generate array to hold solution
    x=[0. for i=1:n]
    #Capture the first value
    x[1]=b[1]/A[1,1]
    #For each equation...
    for i=2:length(b)
        #...solve for x (returns a 1x1 array)...
        tmp=(b[i] - A[i,1:i-1]*x[1:i-1])/A[i,i]
        #...and insert into x
        x[i]=tmp[1]
    end
    return x
end

#Calculate x
x=fsub(A,b)

#Define function to check the solution vector
function csvec(vec1,vec2)
    return all([isapprox(vec1[i],vec2[i]) for i in 1:length(b)])
end

csvec(A*x,b)

true

That was pretty straightforward (once the syntax issues were ironed out).  The algebraic simplicity of this method makes it attractive from a computational standpoint.  The thing is, most matrices in practice do not share this triangular format.  One way to get them there is to use *LU Factorization* to decompose a matrix into upper and lower triangulars.  Then we can use both back and forward substitution, respectively, to solve for the solution vector $x$.  This approach can be used for all square, nonsingular matrices.

There are two phases.  *Factorization* involves the use of Gaussian elimination to factor $A$ into $L$ and $U$.

$$A=LU$$

Note that $L$ will be row-permuted.  Once factored, we are solving the following system.

$$Ax=LUx=L(Ux)=b$$

This arrangement permits a straightforward two-step protocol:

1. Solve for $y$ in $Ly=b$; and,
2. Solve for $x$ in $Ux=y$.

First,let's design a function that performs LU factorization.

In [24]:
#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

function LU_fact(A)
    #Generate containers for LU factors
    L=eye(n)
    U=copy(A)

    #For each column...
    for j=1:n
        #...and each row below the diagonal...
        for i=j+1:n
            #...capture the ratio of the diagonal value to lower row value in that column...
            tmp_ratio=U[i,j]/U[j,j]
            #...store the ratio in L at the lower row value position...
            L[i,j]=tmp_ratio
            #...and then use the ratio as a multiplier when subtracting the diagonal row from the lower row
            #and assign it to the corresponding row in U
            U[i,1:n]=U[i,1:n]-tmp_ratio*U[j,1:n]
        end
    end
    return (L,U)
end

L,U=LU_fact(A)

csvec(L*U,A)

true

Now we solve for $y=Ux$, and ultimately for $x$.  Since $U$ is an upper triangular matrix, we need to use backward substitution in the second stage equation.  Backward substition is just the opposite of forward substitution.  This time, we are operating on an upper triangular matrix.

$$A =
\begin{bmatrix}
a_{11} & a_{12} & a_{13} & \ldots & a_{1n}  \\
        & a_{22} & a_{23} & \ldots & a_{2n}  \\
        &         & \ddots  & \ddots & \vdots   \\
        &         &         & \ddots & a_{n-1,n}\\
  0     &         &         &        & a_{nn}
\end{bmatrix}$$


The first equation starts with the last row.

$$a_{nn}x_n=b_n$$

The second row equation has two unknowns, but one of them was calculated in the first row.  

$$a_{n-1,n}x_n + a_{n-1,n-1}x_{n-1} = b_{n-1}$$

Each new row introduces only one new unknown.

$$x_n = b_n/a_{nn}$$

$$x_{n-1}=(b_{n-1}-a_{n-1,n}x_n)/a_{n-1,n-1}$$

$$x_{n-2}=(b_{n-2}-a_{n-2,n}x_n-a_{n-2,n-1}x_{n-1})/a_{n-2,n-2}$$

$$\vdots$$

$$x_1=(b_1 - a_{1n}x_n - a_{1,n-1}x_{n-1} - \cdots - a_{12}x_2)/a_{11}$$

This progression generalizes to the following expression

$$x_i = (b_i - \sum_{j=i+1}^{n} a_{ij} x_j)/a_{ii}$$

In [73]:
#Define function to solve this system by backward substitution
function bsub(A,b)
    #Capture length of solution vector
    nv=size(A)[1]
    #Generate array to hold solution
    x=[0. for i=1:nv]
    #Capture the last value value
    x[nv]=b[nv]/A[nv,nv]
    #For each equation...
    for i=nv-1:-1:1
        #...solve for x (returns a 1x1 array)...
        tmp=(b[i] - A[i,i+1:nv]*x[i+1:nv])/A[i,i]
        #...and insert into x
        x[i]=tmp[1]
    end
    return x
end

#Generate response vector
b=float(rand(1:10,n))

#Solve for y
y=fsub(L,b)

#Solve for x
x=bsub(U,y)

println("Ly=b:",csvec(L*y,b))
println("Ux=y:",csvec(U*x,y))
println("Ax=b:",csvec(A*x,b))

Ly=b:true
Ux=y:true
Ax=b:true


Now we are well placed to capture an LU solution function all at once.

In [78]:
function LU_solve(A,b)
    #Factor A
    L,U=LU_fact(A)
    #Solve for y
    y=fsub(L,b)
    #Solve for x
    x=bsub(U,y)
    return x
end

#Generate a matrix of random positive integers
A=float(rand(1:10,n,n))

#Generate response vector
b=float(rand(1:10,n))

#Solve for x
x=LU_solve(A,b)

println("Ax=b:",csvec(A*x,b))

Ax=b:true


## Cholesky Factorization

The LU decomposition is a handy method to be sure (and general), but it can be quite expensive computationally.  When the properties of a matrix permit, we can use more efficient methods.  Specifically, when the square matrix $A$ is [Hermitian](https://en.wikipedia.org/wiki/Hermitian_matrix) [positive definite](https://en.wikipedia.org/wiki/Positive-definite_matrix), we can use [Cholesky factorization](https://en.wikipedia.org/wiki/Cholesky_decomposition).

Let's unpack this a bit:

+ A Hermitian matrix is one in which symmetric entries are complex conjugates of each other (that is $a_{ij} = \overline{a_{ji}}$).  Complex conjugates are just numbers that have equivalent real parts and equal but opposite imaginary parts (e.g. 4 & 4, or 4+2$i$ & 4-2$i$).  Symmetric real matrices, therefore, are a subset.
+ A positive, definite matrix is any Hermitian $n \times n$ matrix $M$ for which $z^TMz$ is a positive scalar ($z$ being any real or complex non-zero column vector of length $n$).  [Intuitively](http://math.stackexchange.com/questions/9758/intuitive-explanation-of-a-positive-semidefinite-matrix), any vector multiplied by a positive definite matrix will cannot have it's angle changed by more than 90 degrees.  It is analogous to multiplying a real variable by a positive number.  It can stretch or contract the number, but it cannot reflect it about the origin.

The actual Cholesky decomposition is as follows:

$$A=U^TU$$

$$
\begin{bmatrix}
a_{11} & a_{12} & a_{13} \\
a_{21} & a_{22} & a_{23} \\
a_{31} & a_{32} & a_{33} \\
\end{bmatrix} = 
\begin{bmatrix}
u_{11} & 0 & 0 \\
u_{12} & u_{22} & 0 \\
u_{12} & u_{23} & u_{33} \\
\end{bmatrix}
\begin{bmatrix}
u_{11} & u_{12} & u_{13} \\
0 & u_{22} & u_{23} \\
0 & 0 & u_{33} \\
\end{bmatrix}$$

where $U$ is upper triangular.  The matrix view makes it easier to see what's happening.  The value of $a_{11}$ is straightforward since there is only single value in the column of $U$ and a single value in the row of its transpose:

$$a_{11}=u_{11}u_{11}+0+0$$
$$\rightarrow u_{11} = +\sqrt{a_{11}}$$

In fact, once we have $u_{11}$ the entire first row can be calculated in a similar manner.

$$a_{1j}=u_{11}u_{1j}+0+0$$
$$\rightarrow u_{1j} = \frac{a_{1j}}{u_{11}}$$

Once the first row of $U$ is known, we can sub those values in to the second row equations:

$$u_{2j} = \frac{a_{2j} - u_{12}u_{1j}}{u_{22}}$$

Once this decomposition has been performed, we proceed in much the same way as we did with the LU factorization.

$$Ax = U^TUx = U^T(Ux) = U^Ty = b$$

$$Ux = y$$